In [36]:
# Inspiration: https://www.databricks.com/blog/LLM-auto-eval-best-practices-RAG

In [37]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import pickle
from statistics import mean

In [ ]:
BREADTH_EVAL_PROMPT = """
Please act as an impartial judge and evaluate the quality of the provided answer which attempts to answer the provided question based on a provided context.

What percentage of the key details in the INITIAL SUMMARY is captured by the FINAL SUMMARY? Note that not all information present in INITAL SUMMARY must be in FINAL SUMMARY. Justify the percentage. Format responses in the following way:
SCORE: [percentage]
JUSTIFICATION: [justification]

"""

load_dotenv()

In [40]:
responses = []
with open('generated_responses.pickle', 'rb') as handle:
    responses = pickle.load(handle)

{"Past cases indicate that when arguing against a technology company, key concerns include establishing Article III standing and demonstrating a statutory injury. It has been found that an invasion of privacy alone is insufficient to meet the statutory injury requirements. In the case of Jones v. Ford Motor Company, the court dismissed a class action because the plaintiffs failed to plead a statutory injury under the Washington Privacy Act (WPA). The court emphasized that the WPA requires an injury to one's business, person, or reputation, and an invasion of privacy alone is not enough. This ruling is consistent with other cases in the circuit, such as Brinkley v. Monterey Fin. Servs., LLC. Therefore, it is crucial to carefully analyze the specific statutory requirements and establish a concrete injury to successfully argue against a technology company. The cases cited, Jones v. Ford Motor Company and Brinkley v. Monterey Fin. Servs., LLC, are relevant to the user's query as they provi

In [42]:
client = OpenAI(api_key=os.getenv("OPEN_AI_KEY"))

for final_response in responses:
    curr_response_scores = []
    for summary in responses[final_response]:
        if 'NO RESPONSE' in summary:
            continue
        prompt = f"""TASK: {BREADTH_EVAL_PROMPT}
        
        INITIAL SUMMARY: {summary}
        FINAL SUMMARY: {final_response}
        """
    
        messages = []

        messages.append({"role": "user", "content": prompt})
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages
            )
        chat_message = response.choices[0].message.content
        curr_score = chat_message[chat_message.index("SCORE: ") + 7: chat_message.index("%")]
        curr_response_scores.append(int(curr_score))
    print(final_response)
    print(mean(curr_response_scores))
    print("------")

60
